---
title: Implementing a simple logging system in C++ - 1
tags: 小书匠,logging,log,logger,log4cplus,c++
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# Implementing a simple logging system - 1 in C++

We implement the design in [ 1 ][1].

## LogLevel

In [1]:
%%file LogLevel.h
// LogLevel.h
#pragma once
namespace log {
	enum LogLevel {
		WARN = 200,
		INFO = 100,
		DEBUG = -1,
	};
};

Overwriting LogLevel.h


## LoggingEvent

In [2]:
%%file LoggingEvent.h
// LoggingEvent.h
#pragma once
#include <string>
#include "LogLevel.h"

namespace log
{
	class LoggingEvent
	{
	public:
		std::string message;
		int lineno;
		LogLevel level;
		std::string filename;
		LoggingEvent(const std::string &message,
			     int lineno,
			     const std::string& filename,
			     LogLevel level = LogLevel::INFO)
			      : message(message), lineno(lineno), filename(filename), level(level) {}
	};
};

Overwriting LoggingEvent.h


## Layout

In [9]:
%%file Layout.h

#pragma once
#include <iostream>
#include "LoggingEvent.h"
#include <string>

namespace log
{
	class Layout {
		public:
			virtual std::string format(const LoggingEvent& event) const = 0;
	};

	class SimpleLayout: public Layout
	{
		public:
			std::string format(const LoggingEvent& event) const override
			{
				return "filename: " + event.filename + " - lineno: " + std::to_string(event.lineno) + " - " + event.message;
			}
	};
};

Overwriting Layout.h


## Appender

In [4]:
%%file Appender.h
// Appender.h
#pragma once
#include "LoggingEvent.h"
#include "LogLevel.h"
#include <iostream>
#include "Layout.h"
#include <vector>
#include <memory>

namespace log
{
	class Appender
	{
	public:
		virtual void append(LoggingEvent &) = 0;
		virtual void setLayout(std::shared_ptr<Layout> &layout) = 0;
		Appender(LogLevel level = LogLevel::INFO,
			 std::shared_ptr<Layout> layoutPtr = NULL) : level(level), layoutPtr(layoutPtr) {}

	protected:
		LogLevel level;
		std::shared_ptr<Layout> layoutPtr;
	};

	class ConsoleAppender : public Appender
	{
	public:
		ConsoleAppender() : Appender(LogLevel::INFO, std::make_shared<SimpleLayout>()) {}
		void append(LoggingEvent &event) override
		{
			std::cout << layoutPtr->format(event) << std::endl;
		};
		void setLayout(std::shared_ptr<Layout> &layout) override
		{
			layoutPtr = layout;
		}
	};
}

Overwriting Appender.h


## Logger

In [5]:
%%file Logger.h
// Logger.h
#pragma once
#include "Appender.h"
#include "LoggingEvent.h"
#include <vector>

namespace log
{
	class Logger
	{
		typedef std::shared_ptr<Appender> AppendPtr;
		typedef std::vector<AppendPtr> AppenderList;
		public:
			Logger(Logger* parent=NULL, LogLevel level=LogLevel::INFO): parent(parent), level(level) {}
			void addAppender(AppendPtr appender)
			{
				appenderList.push_back(appender);
			}
			void log(const std::string& message, int lineno, const std::string& filename, LogLevel level)
			{
				if (this->level > level) return ;
				LoggingEvent event(message, lineno, filename, level);
				for (auto& appender: appenderList)
				{
					appender->append(event);
				}
			}
			void setLevel(LogLevel level) {
				this->level = level;
			}
			LogLevel getLevel(LogLevel level) const
			{
				return level;
			}
		private:
			LogLevel level;
			Logger* parent;
			AppenderList appenderList;

	};
};

Overwriting Logger.h


## log.h

To make the logging system easy to use, we provide some macros in `log.h`. Here we referenced [C++ 简单测试框架][2]

In [33]:
%%file log.h
#include "LogLevel.h"
#define LOG(logger, message, LEVEL)  logger.log(message, __LINE__, __FILE__, log::LEVEL)
#define LOG_INFO(logger, message) LOG(logger, message, INFO)
#define LOG_DEBUG(logger, message) LOG(logger, message, DEBUG)
#define LOG_WARN(logger, message) LOG(logger, message, WARN)

Overwriting log.h


## main.cpp

In [34]:
%%file main.cpp
// main.cpp
#include "Appender.h"
#include "Logger.h"
#include "log.h"
#include <iostream>

int main()
{
	std::shared_ptr<log::Appender> appender = std::make_shared<log::ConsoleAppender>();
	log::Logger logger;
	logger.addAppender(appender);
    std::cout << "default level INFO" << std::endl;
	LOG_WARN(logger, "warn");
	LOG_INFO(logger, "info");
	LOG_DEBUG(logger, "debug");
    std::cout << "setLevel WARN" << std::endl;
	logger.setLevel(log::LogLevel::WARN);
	LOG_WARN(logger, "warn");
	LOG_INFO(logger, "info");
	LOG_DEBUG(logger, "debug");
    std::cout << "setLevel DEBUG" << std::endl;
	logger.setLevel(log::LogLevel::DEBUG);
	LOG_WARN(logger, "warn");
	LOG_INFO(logger, "info");
	LOG_DEBUG(logger, "debug");
	return 0;
}

Overwriting main.cpp


In [35]:
%%bash

g++ main.cpp -o main
./main

default level INFO
filename: main.cpp - lineno: 13 - warn
filename: main.cpp - lineno: 14 - info
setLevel WARN
filename: main.cpp - lineno: 18 - warn
setLevel DEBUG
filename: main.cpp - lineno: 23 - warn
filename: main.cpp - lineno: 24 - info
filename: main.cpp - lineno: 25 - debug


# References
1. [The Design of a simple logging system][1]
2. [C++ 简单测试框架][2]

[1]: evernote:///view/22483756/s54/7e132c93-c5fa-410a-baf3-b55fab047f6b/7e132c93-c5fa-410a-baf3-b55fab047f6b
[2]: evernote:///view/22483756/s54/48e89f0f-33bd-4bf4-a9b2-61eba2564453/48e89f0f-33bd-4bf4-a9b2-61eba2564453